DOING ALL THE IMPORTS

In [25]:
from dotenv import load_dotenv
import os 
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

ADDING LLM

In [3]:
load_dotenv()
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature = 0,
    api_key = os.getenv("GOOGLE_API_KEY")
)

Loading pdf and extracting text for embedding

In [ ]:
pdf_path = r'C:\AYUSH\comet\Comet_AI_Assistance\AyushSingh_AIMl_Resume_2026_compressed.pdf'
loader = PyPDFLoader(pdf_path)
docs = loader.load()
#add in text 
text = ""
for doc in docs:
    text += doc.page_content + "\n"
print(text)


Ayush Singh 
ayushrajput8252@gmail.com | +917209602122 | linkedin.com/in/ayush-singh 
 
Summary 
GenAI / AI-ML Engineer specializing in RAG pipelines, LLM applications, LangChain/LangGraph workflows, and 
AI Agents. Built production-style RAG systems, OCR-based verification automation, and transfer learning models 
using Python, vector databases, FastAPI, and Docker. Hackathon winner (Top 10/1000 teams) with strong 
problem-solving background (1000+ DSA problems solved). 
Education 
Parul University, B.Tech in Computer Science 2022 – 2026 
Experience 
Capgemini, Pre-Joining Technical Training Program Jan 2026 – Present 
• Selected through campus recruitment and received an Offer of Intent (LOI). 
• Currently undergoing structured pre-joining training in DSA, OOP, debugging, and coding 
best practices. 
• Solving company-style coding assignments with focus on edge cases and time complexity. 
Skills 
Languages : Java, C++, Python                                                           

TEXT EXTRACTION BASED ON HEADING

In [21]:
import re

HEADINGS = [
    "SUMMARY", "OBJECTIVE",
    "SKILLS", "TECHNICAL SKILLS",
    "EDUCATION",
    "EXPERIENCE", "WORK EXPERIENCE",
    "PROJECTS", "PROJECT",
    "CERTIFICATIONS",
    "ACHIEVEMENTS",
    "INTERNSHIPS",
    "LANGUAGES"
]

def split_heading(text):
    sections = {"OTHER" : ""}
    current = "OTHER"

    for line in text.split("\n"):
        line = line.strip()

        if not line :
            continue

        normalized = re.sub(r"[^A-Za-z]", "", line).upper().strip()

        if normalized in HEADINGS:
            current = normalized
            sections[current] = ""
        else:
            sections[current] += line + "\n"

    return sections

INITIAL BASIC CHECKS

In [23]:
sections = split_heading(text)

count = 0

if sections.get("SKILLS") or sections.get("TECHNICAL SKILLS"):
    count += 1

if sections.get("SUMMARY") or sections.get("OBJECTIVE"):
    count += 1

if sections.get("EXPERIENCE") or sections.get("WORK EXPERIENCE"):
    count += 1

if sections.get("INTERNSHIPS"):
    count += 1

if sections.get("PROJECTS") or sections.get("PROJECT"):
    count += 1


if count >= 4:
    print("Your Resume Contains All necessary info")
else:
    print("Some field missing")


Your Resume Contains All necessary info


SENDING SUMMARY AND SKILLS AND TELLING IF THEY ALLIGNS OR NOT

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# 1) title from summary
prompt1 = ChatPromptTemplate.from_template(
    "Act like a professional IT recruiter. Read this summary:\n{summary}\n"
    "Give exactly one job title that matches it. "
    "Output must be lowercase and max 2 words."
)

chain1 = prompt1 | llm | parser
result1 = chain1.invoke({"summary": sections.get("SUMMARY", "")})

# 2) title from skills
prompt2 = ChatPromptTemplate.from_template(
    "Act like a professional IT recruiter. Read these skills:\n{skills}\n"
    "Give exactly one job title that matches it. "
    "Output must be lowercase and max 2 words."
)

chain2 = prompt2 | llm | parser
result2 = chain2.invoke({"skills": sections.get("SKILLS", "")})

print("Summary Title:", result1)
print("Skills Title:", result2)


Summary Title: genai engineer
Skills Title: ai engineer


In [32]:
prompt3 = ChatPromptTemplate.from_template(
    "Compare these two job titles:\n"
    "1) {t1}\n"
    "2) {t2}\n\n"
    "Return similarity score between 0 and 1 (only number)."
)
chain3 = prompt3 | llm | parser
score = float(chain3.invoke({"t1" : result1, "t2": result2}))
print(score)
if score > 0.8:
    print("GOOD")
else:
    print("Change skill and summary")


0.95
GOOD


Now do a web search through duckduck go or travily extract insignt about this
1. this job tiltle
2. question not genric but from interview experiences 
3. user get 2 option 1. free to chat 2. live vc which give marks on the basis of voice modulalation, clarity, how good he spoke, sound, relevancy and more

In [33]:
from tavily import TavilyClient
api_key = os.getenv("TAVILY_API_KEY")
client = TavilyClient(api_key=api_key)
#here result2 is from skills
title = result2
query = f"{title} placement roadmap trends for 2026"

response = client.search(
    query = query,
    search_depth="advanced",
    max_results = 4,
    include_answer=True,
    include_raw_content=True
)
print(response["answer"])

MissingAPIKeyError: No API key provided. Please provide the api_key attribute or set the TAVILY_API_KEY environment variable.